In [ ]:
import random 
import matplotlib.pyplot
from pulp import *

#Data 
myyntihinta = 1.20
tuotantokustannus = 0.9
kysynta_keskiarvo = 20000
kysynta_keskihajonta = 4000

#Simulointien lukumäärä
simuloinnit = 1000

#Simuloidaan kysyntärealisaatiot
kysynta=[]
random.seed(42)
for i in range(simuloinnit):
    kysynta.append(random.normalvariate(kysynta_keskiarvo,kysynta_keskihajonta))

#Piirretään kysyntäjakauma
matplotlib.pyplot.figure(0)
matplotlib.pyplot.hist(kysynta, color = 'blue', edgecolor = 'black', bins = 100) 
matplotlib.pyplot.title('Kysyntäjakauma')
matplotlib.pyplot.xlabel('Kysyntä (lkm)')
matplotlib.pyplot.ylabel('Frekvenssi')

#Luodaan (minimoiva) lineaarinen optimointimalli PuLP:ssa
malli = LpProblem("Tilauspäätösmalli", LpMaximize)

#Luodaan päätösmuuttuja tilausmäärälle
tilaus = LpVariable("Tilausmäärä", lowBound=0)
#Luodaan "päätösmuuttujat" tuotolle jokaisessa kysyntärealisaatiossa
tuotto_pm={}
for i in range(simuloinnit):
    tuotto_pm[i] = LpVariable("Tuotto_kysyntärealisaatiossa "+str(i+1))
    

#Lisätään kohdefunktio 
malli += (lpSum([tuotto_pm[i] for i in range(simuloinnit)])/simuloinnit, "Odotettu_tuotto")

#Lisätään lineaariset rajoitteet, jotka takaavat, että optimiratkaisussa kussakin kysyntärealisaatiossa tuottopäätösmuuttujien arvo vastaa todellista tuottoa
for i in range(simuloinnit):
    malli += (tuotto_pm[i]<=myyntihinta*tilaus-tuotantokustannus*tilaus)
    malli += (tuotto_pm[i]<=myyntihinta*kysynta[i]-tuotantokustannus*tilaus)

malli.solve()

#Luetaan optimiratkaisun tuotot eri kysyntärealisaatioissa
tuotto=[]
for i in range(simuloinnit):
    tuotto.append(tuotto_pm[i].varValue)

#Piirretään tuottojakauma
matplotlib.pyplot.figure(1)
matplotlib.pyplot.hist(tuotto, color = 'red', edgecolor = 'black', bins = 100) 
matplotlib.pyplot.xlabel('Tuotto (euroa)')
matplotlib.pyplot.ylabel('Frekvenssi')

#Lasketaan odotettu tuotto
odotettu_tuotto = sum(tuotto)/simuloinnit

#Lasketaan tuotojakaumalle CVaR-10%-riskimitta
tuotto.sort()  #Järjestetään suurimmasta pienimpään
pienimmat_tuotot = tuotto[0:int(simuloinnit*0.1)] #Otetaan pienin 10% tuotoista
tuoton_cvar = sum(pienimmat_tuotot)/len(pienimmat_tuotot) #Lasketaan niiden keskiarvo

#Tulostetaan odotettu tuotta ja tuoton CVaR
matplotlib.pyplot.title("Tuottojakauma kun tilausmäärä on "+str(tilaus.varValue)+" (odotettu tuotto="+str(int(odotettu_tuotto))+", CVaR="+str(int(tuoton_cvar))+")")
